# 0. Environment preparation


Let's create SparkSession object - entry point for all Spark computations. We're also loading some data and saving them as views.

In [ ]:
import env_setup
import pyspark.sql.functions as f

spark = env_setup.getSession(local=True)

We can assign each part of a query to a variable. __table()__ method will return a Dataframe which contains a structured dataset. To see what's inside it we can print its schema.

In [ ]:
sales_df = spark.table("sales")
item_prices_df = spark.table("item_prices")

print("# schema of both tables")
sales_df.printSchema()
item_prices_df.printSchema()


We can also print first rows with __show()__

In [ ]:
print("# sample results from both tables")
sales_df.show()
item_prices_df.show()

Dataframe transformations are evaluated lazily. If we chain multiple operations together they will be invoked only when we call an action. To see what's the current *execution plan* we can call __explain()__ method. It will work even for a simple selection from a view like here. 

In [ ]:
print("# query execution plan for this simple select")
sales_df.explain()
item_prices_df.explain()

It prints a one-step operation, which is a csv FileScan, which is correct, because our table was read from such file. We can also see the schema along with some other information.

# 1. SQL support

Spark SQL's name is rather intuitive - we can use Spark to execute SQL queries on our Dataframes.

In [ ]:
sql_df = spark.sql('select item_id, transaction_date from sales where shop_id = "SHOP_1" order by transaction_date desc')
sql_df.show()

### ex1. Use plain SQL query select all transactions with quantity between 2 and 4 (inclusive). Show all results.


### ex2. Print mean unit price for all items

# 2. Dataframe operations

For analysts usinq plain SQL API may be enough, but its much better to use method invocation and chaining to transform out Dataframes. Almost every operation in SQL can be translated to some methods.

In [ ]:
string_df = sales_df.select("item_id", "transaction_date")\
    .filter(f.col("shop_id") == "SHOP_1")\
    .orderBy(f.col("transaction_date").desc())
string_df.show()

 We can use strings to specify column names, but there is another, more dynamic, option to treat columns as fields in a dataframe object.

In [ ]:
field_df = sales_df.select(sales_df.item_id, sales_df.transaction_date)\
    .filter(sales_df.shop_id == "SHOP_1")\
    .orderBy(sales_df.transaction_date.desc())
field_df.show()

The results are the same and actually all these queries will be executed in __exactly__ the same way by Spark. The only difference is the query translation step. Afterwards, when spark has an execution plan of a query it will treat it in the same way regardless of used API. To verify that let's see physical plans of all three queries.

In [ ]:
print("Plain SQL API execution plan")
sql_df.explain()
print("\n Columns as strings approach")
string_df.explain()
print("\n Columns as fields solution")
field_df.explain()

### ex3. Rewrite query from ex1 to dataframe operations

# 3. Joins

Most of the complex queries in relational databases require joins. Spark SQL have them as well.

In [ ]:
print("# joins using plain SQL queries")
spark.sql('select * from sales join item_prices on sales.item_id = item_prices.item_id').show()

print("# using Dataframe API - duplicated item_id column!")
sales_df.join(item_prices_df, sales_df.item_id == item_prices_df.item_id, "inner").show()

print("# dropping redundant column")
sales_with_unit_prices_df = sales_df\
    .join(item_prices_df, sales_df.item_id == item_prices_df.item_id)\
    .drop(sales_df.item_id)
    
sales_with_unit_prices_df.show()

### ex4. Filter out excluded items

Given a Spark Dataframe with a column of items select all transactions from sales_df not containing any of these items.

In [ ]:
print("# Dataframe with column of items we would like to exclude")
excluded_items_df = spark.createDataFrame([("ITEM_2",),("ITEM_4",)], ['item'])
excluded_items_df.show()


# 4. Adding columns

We might want to add a column in dataframe based on values from other columns. __withColumn()__ method is just for that.

In [ ]:
total_sales_df = sales_with_unit_prices_df\
    .withColumn("total_sales", f.col("qty") * f.col("unit_price"))

print("# Added new total_sales column which is a multuply of unit_price and qty")
total_sales_df.show()

Apart from simple ooperations we may use complex predicates while calculating the new value

In [ ]:
print("# Adding price category column based on a condition")
sales_with_transaction_category = total_sales_df\
    .withColumn("transaction_price_category", \
                f.when(f.col("total_sales") > 150, "High")\
                .when(f.col("total_sales") < 60, "Low")\
                .otherwise("Medium"))

sales_with_transaction_category.show()

### ex5. Two-packs of items
We want to create two-packs of items, but sum of their prices must be lower than 360.
hint: use cross join, and alias


# 5. Simple aggregations

We already saw a simple aggregation when calculating mean of prices. Dataframe API allows us to make do it as well.

In [ ]:
print("# using alias to have a better column name")
total_sales_df\
    .groupBy("shop_id")\
    .agg(f.sum(total_sales_df.total_sales).alias("sales"))\
    .orderBy(f.col("sales").desc())\
    .show()
    # .orderBy(total_sales_df.sales) won't work as total_sales_df has no sales column (we define it later)

### ex6. Aggregating data to lists
Produce a column with list of all shops where each item was sold, new column should be named "shops"
hint: check collect_list function

# 6. Date handling

Lots of datasets contain some kind of notion of date or time. Let's see how can we transform it. Our total_sales_df contains *transaction_date* column, we are going to extract each bit out of it with functions from pyspark.sql.functions module

In [ ]:
print("# extracting multiple elements of date")
total_sales_df\
    .withColumn("year", f.year(f.col("transaction_date")))\
    .withColumn("month", f.month(f.col("transaction_date")))\
    .withColumn("day", f.dayofmonth(f.col("transaction_date")))\
    .withColumn("day_of_year", f.dayofyear(f.col("transaction_date")))\
    .withColumn("day_of_week", f.date_format(f.col("transaction_date"), 'u'))\
    .withColumn("day_of_week_str", f.date_format(f.col("transaction_date"), 'E'))\
    .withColumn("week_of_year", f.weekofyear(f.col("transaction_date")))\
    .show()

We don't need to define new column to use obtained values. Here's an example of getting sales aggregated by week.

In [ ]:
print("# aggregate sales by week")
total_sales_df\
    .groupBy(f.weekofyear(f.col("transaction_date")))\
    .agg(f.sum(f.col("total_sales")))\
    .show()

### ex7. Weekly sales aggregation not starting on Monday
For Spark, each week starts on Monday. But what if we want to start aggregation on a different day, for example Sunday?

### ex8. (Homework) Sales aggregation with preserving date
Sometimes we want to preserve the date of the week (for example last day) instead of year week number. Try implementing aggregation above where instead of week number there is a date of last day of given week. Try to do it without using join. 

hint: maybe *next_day()* function will be helpful.

# 7. Using results of one query in another

How can we get results from one query to another? We could use joins, but there are other ways. Let's say we want to add maximal price of all items to each sales row.

In [ ]:
print("# Calculate global max unit_price")
item_prices_df\
    .select(f.max(item_prices_df.unit_price).alias("max_price"))\
    .show()

But how can we get that value out of the dataframe? If we only have one Row then using *first()* method will be enpough to return a Row object. In cases we have more rows then we need to use *collect()*.
Both of these methods are actions, which means they will invoke calculations. Furthermore, result of these operations will be sent directly to driver. This can be be problematic for large Dataframes.

In [ ]:
max_date_row = item_prices_df\
    .select(f.max(item_prices_df.unit_price).alias("max_price"))\
    .first() # first() returns first Row, collect returns list of rows
    #.collect()[0]

print(max_date_row)

We ended up with a Row object. It has a field for each column in the Dataframe. We can extract values either by index or by name.

In [ ]:
print(max_date_row[0])
print(max_date_row.max_price)
print(max_date_row['max_price'])

max_price = max_date_row.max_price

Now to include it in each sales rows, we need to add a new column of literal value using __lit()__ function.

In [ ]:
print("# adding it as a literal (constant) column")
sales_with_max_global_price_df = total_sales_df\
    .withColumn("global_max_price", f.lit(max_price))

sales_with_max_global_price_df.show()

### ex8. Adding constant column using cross join
Most of the times we want to avoid unnnecessary actions. They break the flow of a query, canot be optimized and require sending data over network to driver and, in our case, back again. Let's implement query above using cross join.

You must make sure that Dataframe used in cross join has at most couple elements, if not then the number of rows will explode.

# 8. Window functions

To perform partial aggregations but preserving initial number of rows we could use joins.Let's try that to get latest transaction date for each shop.

In [ ]:
max_date_by_store_df = total_sales_df\
    .groupBy(f.col("shop_id"))\
    .agg(f.max("transaction_date").alias("max_transaction_date_by_shop")) 
    
total_sales_df.join(max_date_by_store_df, ["shop_id"])\
    .show() # 

__Side note__: instead of join condition we passed a list with one column name. This way we avoid duplicationd of that column in join.

Another option is using __window functions__. They are experimental since spark 1.4, but they are widely used in production.

First, we need to define Window with a grouping column using __partitionBy()__ method. Then we create a new column as usual, but after invoking a function we add __.over(window)__ to say that this is not a global operation.

In [ ]:
from pyspark.sql import Window

window = Window.partitionBy(f.col("shop_id"))

total_sales_df\
    .withColumn("max_transaction_date_by_shop", f.max(f.col("transaction_date")).over(window)).show()


In each partition partition we can specify order in which we traverse the rows. Let's use that to find ordinals for transactions for each item.

In [ ]:
window_by_item_sorted = Window.partitionBy(f.col("item_id")).orderBy(f.col("transaction_date"))

total_sales_df\
    .withColumn("item_transaction_ordinal", f.rank().over(window_by_item_sorted))\
    .show()

We can specify also how large the window should be in each step using **rowsBetween()** method. We can use it to find average price from last two transactions in given shop, ordered by transaction date (like a group-level, moving average)

In [ ]:
window_by_transaction_date = Window\
    .partitionBy(f.col("shop_id"))\
    .orderBy(f.col("transaction_date"))\
    .rowsBetween(-1,Window.currentRow)

total_sales_df\
    .withColumn("price_moving_average", f.mean(f.col("total_sales")).over(window_by_transaction_date))\
    .orderBy(f.col("shop_id"), f.col("transaction_date"))\
    .show()

### ex9. Cumulative moving average of quantities
Find average quantity of items from current and all previous transactions for given item ordered by transaction date.

# 9. Complex aggregations

Sometimes we need more complex aggregations. Let's say we want to analyse weekly sales in each shop. We would like to get a Dataframe with one row per shop and a list of all transactions with week and year numbers.
We could try doing that using multiple invocations of collect_list function.

In [ ]:
weekly_sales_by_shop_df = total_sales_df\
    .groupBy("shop_id", f.weekofyear("transaction_date").alias("week"), f.year("transaction_date").alias("year"))\
    .agg(f.sum("total_sales").alias("sales"))

print("# Sales Dataframe with week and year columnns")
weekly_sales_by_shop_df.show()
        
print("# aggregating sales with three collect_list invocations")
shop_sales_weekly_series_df = weekly_sales_by_shop_df\
    .groupBy("shop_id")\
    .agg(f.collect_list("week"),f.collect_list("year"),  f.collect_list("sales"))

shop_sales_weekly_series_df.show(truncate=False)

Unfortunately solution above won't work correctly, as ordering in each column may be different. Passing list of columns to colllect_list won't work either as there is no such API: .agg(f.collect_list(["sales", "week"]))  

We can overcome that using a struct method which aggregates values for each row in a structure (similar to dict)

In [ ]:
shop_sales_weekly_series_df = weekly_sales_by_shop_df\
    .groupBy("shop_id")\
    .agg(f.collect_list(f.struct(["year", "week", "sales"])).alias("sales_ts"))

shop_sales_weekly_series_df.show(truncate=False)
shop_sales_weekly_series_df.printSchema()

Ok, we have a time series for each shop, but what if we want to have it ordered by date? We could try sorting the dataframe before aggregation. Unfortunately Spark doesn't preserve this ordering after groupBy

In [ ]:
ordered_weekly_sales_df = weekly_sales_by_shop_df\
    .orderBy("shop_id", "year", "week")
  
ordered_weekly_sales_df.show()

wrongly_sorted_series_df = ordered_weekly_sales_df\
    .groupBy("shop_id")\
    .agg(f.collect_list(f.struct(["year", "week", "sales"])).alias("sales_ts"))
    
wrongly_sorted_series_df.show(truncate=False)

# 10. Defining custom UDFs

To solve that issue we need to sort the time series after aggregation. To do that we need to define a custom User Defined Functions (UDF).
Such function is invoked on each row separately. It can take as many columns as we need. It can contain any custom Python code, even from libraries available in your environment. 

Let's create a function appending a custom prefix to value from another column

In [ ]:
def my_custom_function(column1):
    return "AFTER_UDF_" + str(column1)

my_custom_udf = f.udf(my_custom_function)
df_after_udf = shop_sales_weekly_series_df.withColumn("sales_ts_after_udf", my_custom_udf(f.col("sales_ts")))
df_after_udf.show()
print("# Schema of the new dataframe")
df_after_udf.printSchema()

To use custom UDF in a plain, SQL query we need to register it in a sqlContext

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sqlContext = SQLContext(spark.sparkContext)
sqlContext.registerFunction("my_udf", my_custom_function)

spark.sql("select my_udf(shop_id) from sales").show()


### ex10. UDF calculating sales for given transaction by multiplying qty and unit_price  
Create a UDF taking two columns (qty and unit_price) from total_sales_df and returning their product as a new column

What happens if we want to return more than one column from a UDF? Let's try returning a tuple.

In [ ]:
def split_shop_id(shop_id):
    s, i = shop_id.split("_")
    return s, int(i) 

split_shop_id_udf = f.udf(split_shop_id)
df_udf_no_schema = shop_sales_weekly_series_df.withColumn("shop_id_splits", split_shop_id_udf(f.col("shop_id")))
print("# Results not as expected - seems like calling toString on Object")
df_udf_no_schema.show(truncate=False)

print("# Actual inferred schema: one string instead of a tuple")
df_udf_no_schema.printSchema()

To avoid that situation we need to define a result schema for our UDF

In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

schema = StructType([StructField("s", StringType()), StructField("i", IntegerType())])
udf_with_schema = f.udf(split_shop_id, schema)

df = df_udf_no_schema.withColumn("shop_id_splits_with_schema", udf_with_schema(f.col("shop_id")))
df.show(truncate=False)
print("# New schema is correct as well")
df.printSchema()

## Creating multiple columns based on a result from UDF
In the last example we created one column with multiple values, now let's try to extract them to separate columns

This can be done using asterisk __\*__

In [ ]:
df_split_shop_id = df.select("*", "shop_id_splits_with_schema.*").drop("shop_id_splits_with_schema")
df_split_shop_id.show()
print("# Schema was updated and new fields have correct types")
df_split_shop_id.printSchema()

Solution above will invoke UDF as many times as new column (it's a feature not a bug! https://issues.apache.org/jira/browse/SPARK-17728").

For costly UDFs (and in pySpark most of them are very costly) we have a workaround: we need to explode an array with one element - result of the UDF

In [ ]:
df_split_shop_id_correct = df_udf_no_schema.withColumn("shop_id_splits_with_schema", \
                                 f.explode(f.array(udf_with_schema(f.col("shop_id")))))

df_split_shop_id_correct = df_split_shop_id_correct \
    .select("*", "shop_id_splits_with_schema.*") \
    .drop("shop_id_splits_with_schema")
df_split_shop_id_correct.show()
print("# Results and schema are the same")
df_split_shop_id_correct.printSchema()


But how do we know that this UDF will be invoked multiple times? Let's take a deeper look at execution plans of both queries.

For the first version we can see:

> +- BatchEvalPython [split_shop_id(shop_id#10), split_shop_id(shop_id#10), split_shop_id(shop_id#10)], [shop_id#10, sales_ts#3899, pythonUDF0#4442, pythonUDF1#4443, pythonUDF2#4444]

which contains multiple pythonUDF references.
                                  
For the updated solution there's only one invocation:

> +- BatchEvalPython [split_shop_id(shop_id#10)], [shop_id#10, sales_ts#3899, shop_id_splits#4155, pythonUDF0#4448]


In [ ]:
df_split_shop_id.explain()
print("\n\n\n")
df_split_shop_id_correct.explain()


### ex.11 Sort each time series in wrongly_sorted_series_df from previous exercise in descending order and compare to initial ts 
tip: use python's sorted method inside a UDF. FloatType and ArrayType imports may be usefull as well